##Requirements
In this secttion we import the required packages for training our classifier.


In [1]:
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
#We mount the google drive. If You are running this notebook locally do nott run this cell. 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Configurations
In this section we define the configs for training. 

In [3]:
DATASET = "Allele B Cropped"
X_PATH = "/content/drive/Shareddrives/Exploding Gradients/X_b.npy"
Y_PATH= "/content/drive/Shareddrives/Exploding Gradients/y_b.npy"


BACKBONE = "resnet50"
MULTI_BACKBONE = True
OPTIM = "Adam"
LR =5e-5
SCHEDULER = "None"
EPOCHS = 40
BATCHSIZE = 4
AUGMENTATION = "None"

#The following is a list of hyper parameters to test. All Permuttations will be
#tested

DROPOUT = [0,0.1,0.2,0.5]
WEIGHT_DECAY = [0,1e-3,1e-5]
FREEZE = [10,25,40,50,55]

#Data Processing
In this section, we read the dataset as a pre saved numpy array. After reading the datset. we divide it into train-testtt sets. We tthen create a pytorch dataset which we will then turn into a dataloader.


In [4]:
#We read the 
x = np.load(X_PATH)
y = np.load(Y_PATH)
y = np.squeeze(y.astype(np.int16))
b = np.zeros((y.size, y.max()+1))
b[np.arange(y.size),y] = 1
y = b
print("X Tensor Shape: ",x.shape)
print("y Tensor Shape: ",y.shape)

X Tensor Shape:  (285, 4, 200, 1024, 3)
y Tensor Shape:  (285, 3)


##Model
Code for tensorflow model.

In [5]:
IMG_SHAPE = (200, 1024, 3)

class Classifier(tf.keras.Model):

    def __init__(self, num_classes=1000):
        super(Classifier, self).__init__()
        self.backbone = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                                       include_top=False,
                                                       weights='imagenet')
        self.classifier= tf.keras.Sequential([
                                                  
                                                   tf.keras.layers.Dense(3)
        ])
       

    def call(self, inputs):

        images = [inputs[:,i,:,:,:] for i in range(4)]
        encodings = [tf.reshape(self.backbone(img),[inputs.shape[0],-1]) for img in images]
        encodings = tf.concat(encodings,1)

        return self.classifier(encodings)



##Training

In [ ]:
model = Classifier()
model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001,),
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True))

model.fit(x[0:232],y[0:232],4,epochs=4,validation_data=(x[232:264],y[232:264]))

Epoch 1/4
58/58 [==============================] - 42s 500ms/step - loss: 4.8105 - val_loss: 3.7167
Epoch 2/4
12/58 [=====>........................] - ETA: 19s - loss: 0.9276